# Homework 6
## Damian Franco
## CS-575

This program implements three version of QR decomposition. The Gram-Schmidt approach is used with Classical Gram-Schmidt, Modified Gram-Schmidt version 1 and Modified Gram-Schmidt version 2. Each section below has the function for its respective approach. Testing is done with a small 3x2 matrix and with a large matrix with linearly dependent columns.

In [15]:
# Importing the required modules
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import time
from prettytable import PrettyTable
import scipy
import scipy.linalg 
from pprint import pprint
%matplotlib inline
import math

In [2]:
# Saves the current plot to desktop since working in Google Colab
from google.colab import files
#plt.savefig("my_plot.png", bbox_inches='tight', dpi=300)
#files.download("my_plot.png")

# Classical Gram-Schmidt

In [3]:
# Classical GS
def qr_cgs(A):
    (m,n) = A.shape
    Q = A.copy()
    R = np.zeros((n,n))
    for j in range(n):
        for i in range(j):
            R[i,j] = np.dot(Q[:,i],A[:,j])
            Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
        R[j,j] = np.linalg.norm(Q[:,j])
        Q[:,j] = Q[:,j]/R[j,j]
       
    return Q, R

In [4]:
# Lets test the algorithms on a simple 2x2 system
A = np.array([ [1.,2.], [3.,4.], [5.,6.] ])

In [23]:
Q_cgs, R_cgs = qr_cgs(A)
print('Q =', Q_cgs)
print('R =', R_cgs)

Q = [[ 0.16903085  0.89708523]
 [ 0.50709255  0.27602622]
 [ 0.84515425 -0.34503278]]
R = [[5.91607978 7.43735744]
 [0.         0.82807867]]


# Modified Gram-Schmidt (Version 1)

In [32]:
# Modified GS - Version 1. 
def qr_mgs_ver_1(A):
    (m,n) = A.shape
    Q = A.copy()
    R = np.zeros((n,n))
    for j in range(n):
        R[j,j] = np.linalg.norm(Q[:,j])
        Q[:,j] = Q[:,j]/R[j,j]
        for i in range(j+1, n):
            R[j,i] = np.dot(Q[:,j], Q[:,i])
            Q[:,i] = Q[:,i] - R[j,i]*Q[:,j]
    return Q, R

In [33]:
Q_ver1, R_ver1 = qr_mgs_ver_1(A)
print('Q =', Q_ver1)
print('R =', R_ver1)

Q = [[ 0.16903085  0.89708523]
 [ 0.50709255  0.27602622]
 [ 0.84515425 -0.34503278]]
R = [[5.91607978 7.43735744]
 [0.         0.82807867]]


# Modified Gram-Schmidt (Version 2)

In [36]:
# Modified GS - Version 2. Notice the similarity to Classical GS.
def qr_mgs_ver_2(A):
    (m,n) = A.shape
    Q = A.copy()
    R = np.zeros((n,n))
    for j in range(n):
        for i in range(j-1):
            R[i,j] = np.dot(Q[:,i], Q[:,j])
            Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
        R[j,j] = np.linalg.norm(Q[:,j])
        Q[:,j] = Q[:,j]/R[j,j]
    return Q, R

In [37]:
Q_ver2, R_ver2 = qr_mgs_ver_2(A)

print('Q =', Q_ver2)
print('R =', R_ver2)

Q = [[0.16903085 0.26726124]
 [0.50709255 0.53452248]
 [0.84515425 0.80178373]]
R = [[5.91607978 0.        ]
 [0.         7.48331477]]


# Test with nearly linearly dependent cols

In [40]:
#Lets test the algorithms on a matrix with nearly linearly dependent columns
m = 200; n = 150
a = np.random.normal(loc=0, scale=1, size=(m, 1))
epsi = 1e-5
b=a@np.ones((1,n))

#The matrix A below has nearly linearly dependent columns
A = np.multiply ((a@np.ones((1,n))), np.ones((m,n))) + epsi*np.random.normal(loc=0, scale=1, size=(m, n))

# Empty lists for norms calculated
normAs = []
normQs = []

# Classical Gram-Schmidt
Q,R = qr_cgs(A)
norm_of_A_minus_QR = np.linalg.norm(A-Q@R)
norm_of_QTQ_minus_I = np.linalg.norm(Q.T@Q - np.identity(n))
normAs.append(norm_of_A_minus_QR)
normQs.append(norm_of_QTQ_minus_I)
print('CGS: ||A-QR|| = ', norm_of_A_minus_QR, '&& ||QTQ-I|| = ',norm_of_QTQ_minus_I)

# Modified Gram-Schmidt Version 1
Q,R = qr_mgs_ver_1(A)
norm_of_A_minus_QR = np.linalg.norm(A-Q@R)
norm_of_QTQ_minus_I = np.linalg.norm(Q.T@Q - np.identity(n))
normAs.append(norm_of_A_minus_QR)
normQs.append(norm_of_QTQ_minus_I)
print('MGS-ver1: ||A-QR||  = ', norm_of_A_minus_QR, '&& ||QTQ-I|| = ',norm_of_QTQ_minus_I)

# Modified Gram-Schmidt Version 2
Q,R = qr_mgs_ver_2(A)
norm_of_A_minus_QR = np.linalg.norm(A-Q@R)
norm_of_QTQ_minus_I = np.linalg.norm(Q.T@Q - np.identity(n))
normAs.append(norm_of_A_minus_QR)
normQs.append(norm_of_QTQ_minus_I)
print('MGS-ver2: ||A-QR||  = ', norm_of_A_minus_QR, '&& ||QTQ-I|| = ',norm_of_QTQ_minus_I)

CGS: ||A-QR|| =  7.459892324685454e-14 && ||QTQ-I|| =  0.00043386787336402015
MGS-ver1: ||A-QR||  =  7.451390748880851e-14 && ||QTQ-I|| =  4.5864016513518555e-10
MGS-ver2: ||A-QR||  =  7.394211914570254e-14 && ||QTQ-I|| =  3.3716815787946195


In [46]:
# Create error table with sizes
# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Version", "||A-QR||", "||QTQ-I||"])
names = ['CGS', 'MGS Ver.1', 'MGS Ver.2']
# Add rows
for i in range(3):
  myTable.add_row([names[i], normAs[i], normQs[i]])

print(myTable)

+-----------+-----------------------+------------------------+
|  Version  |        ||A-QR||       |       ||QTQ-I||        |
+-----------+-----------------------+------------------------+
|    CGS    | 7.459892324685454e-14 | 0.00043386787336402015 |
| MGS Ver.1 | 7.451390748880851e-14 | 4.5864016513518555e-10 |
| MGS Ver.2 | 7.394211914570254e-14 |   3.3716815787946195   |
+-----------+-----------------------+------------------------+
